# Using GeoPandas' ```.sjoin()``` to Count the Number of Points Within a Geometry
### Calculating the number of households within certain distances from tanks for each county

### Import statements

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import geopandas as gpd
import dask
from dask import dataframe as dd
import dask_geopandas
import os

### Setting ```DATA_DIR```
In order to read in files from this repository, we must set ```DATA_DIR``` to be the data folder within this repository. This requires ```os.getcwd()``` to return the path to the processing notebook of this repository, so ```xxx/codeplus-celine-dcc-package/procesing```, where ```xxx``` is the path to where you cloned this repository. If it is not, use ```os.chdir(path)``` to change the current working directory to ```xxx/codeplus-celine-dcc-package/procesing``` before getting the current working directory in ```DATA_DIR = os.getcwd()```, where ```path``` is ```xxx/codeplus-celine-dcc-package/procesing```.

In [3]:
DATA_DIR = os.getcwd()
DATA_DIR = DATA_DIR.replace('processing', 'data')
DATA_DIR

'/hpc/home/at341/ondemand/codeplus-celine-dcc-package/data'

### Reading county shapefile dataframe
To count the number of households within five miles of a storage tank for each county, we use GeoPandas' ```.sjoin()``` method. Using this method, we will perform a spatial join between each county's geometry and the dataframe including Point geometries for each household in the US. For this, we need a dataframe with geometries for all counties in the US- which we took from the United States Census Bureau's Cartographic Boundary Files (available [here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)). Then, we filter to exclude counties from Alaska, Hawaii, Puerto Rico, Virgin Islands, American Samoa, Guam, Northern Marian Islands, as there are no tanks in those regions in the AST dataset. We also drop unnecessary columns.

In [4]:
df_counties = gpd.read_file(DATA_DIR + '/source_files/county_shapefiles/counties.shp')
df_counties = df_counties[((df_counties['STATEFP'] != '02') & (df_counties['STATEFP'] != '15') & 
                          (df_counties['STATEFP'] != '72') & (df_counties['STATEFP'] != '78') & 
                          (df_counties['STATEFP'] != '60') & (df_counties['STATEFP'] != '66') & 
                          (df_counties['STATEFP'] != '69'))]
df_counties = df_counties[['NAME', 'geometry']]
df_counties.rename(columns = {'NAME': 'county'}, inplace = True)
df_counties.head()

,county,geometry
0,Riley,"POLYGON ((-96.96095 39.28670, -96.96106 39.288..."
1,Ringgold,"POLYGON ((-94.47167 40.81255, -94.47166 40.819..."
2,Carbon,"POLYGON ((-109.79867 45.16734, -109.68779 45.1..."
3,Bear Lake,"POLYGON ((-111.63452 42.57034, -111.63010 42.5..."
4,Buffalo,"POLYGON ((-92.08384 44.41200, -92.08310 44.414..."


### Reading InfoUSA Data (pre-processed)
This InfoUSA data was preprocessed to include the distance between each household and the nearest tank. To understand how it was processed in more detail, visit processing notebok **05_all_us_dist_processing**. We need this information to filter for only households within five miles of a storage tank, which we do below.

In [5]:
df_hh = pd.read_parquet(DATA_DIR + '/distances_all_hh.parquet')
df_hh.head()

,zip,county_fips,state,child_num,age_code,lat_h_3857,lon_h_3857,lat_h_4326,lon_h_4326,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_m,distance_mi,distance_category
0,16965,42269,IN,3,C,-8.556472e+06,4.754685e+06,39.230097,-76.864096,7.975933,32.552888,23.825800,45.694335,3.543941,17.889439,21.913723,2.410337e+04,14.977140,4
1,79667,08484,NV,5,C,-1.076073e+07,5.469166e+06,44.024061,-96.665285,6.897670,11.518197,-1.000000,4.252650,-1.000000,9.396389,5.344151,1.307560e+06,812.480090,4
2,88819,35578,ID,1,I,-1.251261e+07,4.094840e+06,34.490381,-112.402712,0.509994,8.385226,5.089364,9.677147,-1.000000,10.415246,5.679496,1.503771e+06,934.400050,4
3,16748,25538,PA,10,K,-9.857755e+06,4.129311e+06,34.745220,-88.553720,1.289903,15.693502,3.403131,19.224199,-1.000000,8.517747,8.021414,8.193696e+05,509.132686,4
4,43449,11049,NJ,1,C,-9.267351e+06,5.493176e+06,44.178941,-83.250028,2.205648,26.709422,5.171142,17.753031,0.000000,8.982879,10.137020,7.857940e+04,48.826977,4


We also only keep the ```lat_h_4326``` and ```lon_h_4326``` columns because these are the coordinates for each household we will use to create the Point geometries for each household in the households GeoDataFrame. We use the coordinates in EPSG 4326 rather than the coordinates in EPSG 3857 because the geometries in the ```df_counties``` GeoDataFrame are in EPSG 4326. The coordinate system must remain consistent in order to get accurate results when performing the spatial join with ```.sjoin()```.

In [6]:
df_hh = df_hh[['lat_h_4326', 'lon_h_4326', 'distance_category']]
df_hh = df_hh[df_hh['distance_category'] != 4]
df_hh.head()

,lat_h_4326,lon_h_4326,distance_category
66795,36.195148,-114.966174,3


### Use Dask to transform pandas dataframe to a geopandas dataframe
To perform a spatial join between two dataframes, each of these dataframes must be GeoDataFrames- thus, we must convert our ```df_hh``` dataframe to a GeoDataFrame. However, as this dataframe has more than 17 million rows, converting it without using Dask is not time-efficient. Hence, we turned to Dask, an open-source Python library for parallel computing. It allows us to efficiently execute the transformation of our dataframe to a GeoDataFrame, even when working with over 17 million rows. 

To use Dask, we first converted our dataframe to a Dask dataframe, using Dask's ```.from_pandas()``` method. This method takes in our pandas dataframe along with the ```npartitions``` parameter, which is used to specify the number of 'sections' the dask dataframe will be split into.

Note: using the test, synthetic InfoUSA data in this repository, we only have around 1 million rows instead of 17 million rows. Dask is still suggested for time efficiency, as that is still a large number of observations.

In [7]:
df_dask = dd.from_pandas(df_hh, npartitions = 500)

Then, we specify what manipulation to the dask dataframe ```df_dask``` to compute. In this case, we use Dask Geopandas' ```.points_from_xy()``` method to convert the pandas dask dataframe into a geopandas dask dataframe.

In [8]:
%%time
df_dask['geometry'] = dask_geopandas.points_from_xy(df_dask, 'lon_h_4326', 'lat_h_4326')

CPU times: user 8.72 ms, sys: 635 µs, total: 9.35 ms
Wall time: 9.22 ms


After, we convert the dask geodataframe into a geopandas dataframe:

In [9]:
%%time
gdf = dask_geopandas.from_dask_dataframe(df_dask)

CPU times: user 9.11 ms, sys: 353 µs, total: 9.46 ms
Wall time: 9.23 ms


Calling compute puts all the above code into action. Dask executes each set of commands on each partition, as specified above. This returns GeoDataFrame ```gdf_hh```, with over 17 million rows, in around five seconds.

In [10]:
%%time
gdf_hh = gdf.compute()

CPU times: user 10.7 ms, sys: 1.63 ms, total: 12.3 ms
Wall time: 11.9 ms


In [11]:
gdf_hh = gdf_hh[['distance_category', 'geometry']]
gdf_hh.head()

,distance_category,geometry
66795,3,POINT (-114.96617 36.19515)


### Computing number of households within 0.5 mi of a storage tank in each county
Now that we have our two GeoDataFrames, we ```.sjoin()``` to find which households are within 0.5 miles from a tank in each county.

#### Filtering the household data for only households within 0.5 mi of a storage tank
First, we created a new GeoDataFrame ```gdf_hh_half_mi``` that included only the households within 0.5 miles of a storage tank.

In [12]:
gdf_hh_half_mi = gdf_hh[gdf_hh['distance_category'] == 1]
gdf_hh_half_mi.head()

,distance_category,geometry


#### Adding column in county dataframe with the number of households within 0.5mi of a tank
Then, we used ```.sjoin()``` to find the number of households within 0.5 miles of a tank for each county.

Calling ```.sjoin()``` and passing in two GeoDataFrames will return a new GeoDataFrame that only includes the observations with geometries that are the intersections of the two original GeoDataFrames. In our case, passing in a GeoDataFrame with the geometry for Harris County and a GeoDataFrame with all the households within 0.5 miles of a tank to the ```.sjoin()``` method returns a new GeoDataFrame with all the households within 0.5 miles of a tank in Harris County, as it returns all the Point geometries that intersects the Harris county Polygon geometry. Computing the length of that new dataframe through ```len()``` returns the number of households within 0.5 miles in Harris county, then.

However, we need to do this for all counties in the US, so we use for loop. This loop iterates every row of ```df_counties``` finds the intersection between that county and the households within 0.5 miles of a tank (```gdf_hh_half_mi```), computes the length of that ```intersect_df```, and adds that number to a new column in ```df_counties```. We add the column to ```df_counties``` specifically, because then we can use this exact dataframe to plot a map of the US where each county is colored by the number of households within five miles of a storage tank, as it contains all the geometries for each county required for a GeoViews visualization.

In [13]:
%%time

df_counties['hh_half_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_half_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_half_mi'].iloc[i] = num

df_counties

CPU times: user 37.1 s, sys: 55.1 ms, total: 37.1 s
Wall time: 37.3 s


,county,geometry,hh_half_mi
0,Riley,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",0
1,Ringgold,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",0
2,Carbon,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",0
3,Bear Lake,"POLYGON ((-111.63452 42.57034, -111.63010 42.5...",0
4,Buffalo,"POLYGON ((-92.08384 44.41200, -92.08310 44.414...",0
...,...,...,...
3229,Asotin,"POLYGON ((-117.47999 46.12199, -117.41948 46.1...",0
3230,Candler,"POLYGON ((-82.25457 32.35150, -82.25276 32.353...",0
3231,Tom Green,"POLYGON ((-101.26763 31.55646, -101.25039 31.5...",0
3232,Licking,"POLYGON ((-82.78181 39.94698, -82.78126 39.955...",0


### Computing number of households within 1 mi of a storage tank in each county
Following the same steps as described above, except filtering for households within one mile of a storage tank.

#### Filtering the household data for only households within 1 mi of a storage tank
Note: using the test, synthetic data there are no households within 1 mi of a storage tank. More households would appear with the original InfoUSA and AST datasets.

In [14]:
gdf_hh_one_mi = gdf_hh[gdf_hh['distance_category'] == 2]
gdf_hh_one_mi

,distance_category,geometry


#### Adding column in county dataframe with the number of households within 1 mi of a tank
Takes around 30min

In [15]:
%%time

df_counties['hh_one_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_one_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    # print(num)
    
    # adding num to df_counties column
    df_counties['hh_one_mi'].iloc[i] = num

df_counties

CPU times: user 37.4 s, sys: 44.6 ms, total: 37.5 s
Wall time: 37.7 s


,county,geometry,hh_half_mi,hh_one_mi
0,Riley,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",0,0
1,Ringgold,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",0,0
2,Carbon,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",0,0
3,Bear Lake,"POLYGON ((-111.63452 42.57034, -111.63010 42.5...",0,0
4,Buffalo,"POLYGON ((-92.08384 44.41200, -92.08310 44.414...",0,0
...,...,...,...,...
3229,Asotin,"POLYGON ((-117.47999 46.12199, -117.41948 46.1...",0,0
3230,Candler,"POLYGON ((-82.25457 32.35150, -82.25276 32.353...",0,0
3231,Tom Green,"POLYGON ((-101.26763 31.55646, -101.25039 31.5...",0,0
3232,Licking,"POLYGON ((-82.78181 39.94698, -82.78126 39.955...",0,0


### Computing number of households within 5mi of a storage tank in each county
Following the same steps as described above, except filtering for households within five miles of a storage tank. Because the households GeoDataFrame has over 14 million rows, this process takes quite a while.

#### Filtering the household data for only households within 5 mi of a storage tank

In [16]:
gdf_hh_five_mi = gdf_hh[gdf_hh['distance_category'] == 3]
gdf_hh_five_mi

,distance_category,geometry
66795,3,POINT (-114.96617 36.19515)


#### Adding column in county dataframe with the number of households within 5 mi of a tank
Takes around 4 hours.

In [17]:
%%time 

df_counties['hh_five_mi'] = 0
df_counties_temp = df_counties

for i in range(0, len(df_counties)):
    county = df_counties_temp.head(n=1)
    intersect_df = gpd.sjoin(county, gdf_hh_five_mi, how='inner', predicate='intersects')
    df_counties_temp = df_counties_temp.iloc[1:, :]
    num = len(intersect_df)
    
    # adding num to df_counties column 
    df_counties['hh_five_mi'].iloc[i] = num

df_counties

CPU times: user 35.1 s, sys: 122 ms, total: 35.2 s
Wall time: 35.3 s


,county,geometry,hh_half_mi,hh_one_mi,hh_five_mi
0,Riley,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",0,0,0
1,Ringgold,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",0,0,0
2,Carbon,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",0,0,0
3,Bear Lake,"POLYGON ((-111.63452 42.57034, -111.63010 42.5...",0,0,0
4,Buffalo,"POLYGON ((-92.08384 44.41200, -92.08310 44.414...",0,0,0
...,...,...,...,...,...
3229,Asotin,"POLYGON ((-117.47999 46.12199, -117.41948 46.1...",0,0,0
3230,Candler,"POLYGON ((-82.25457 32.35150, -82.25276 32.353...",0,0,0
3231,Tom Green,"POLYGON ((-101.26763 31.55646, -101.25039 31.5...",0,0,0
3232,Licking,"POLYGON ((-82.78181 39.94698, -82.78126 39.955...",0,0,0


In [18]:
df_counties['hh_total'] = df_counties['hh_half_mi'] + df_counties['hh_one_mi'] + df_counties['hh_five_mi']
df_counties

,county,geometry,hh_half_mi,hh_one_mi,hh_five_mi,hh_total
0,Riley,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",0,0,0,0
1,Ringgold,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",0,0,0,0
2,Carbon,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",0,0,0,0
3,Bear Lake,"POLYGON ((-111.63452 42.57034, -111.63010 42.5...",0,0,0,0
4,Buffalo,"POLYGON ((-92.08384 44.41200, -92.08310 44.414...",0,0,0,0
...,...,...,...,...,...,...
3229,Asotin,"POLYGON ((-117.47999 46.12199, -117.41948 46.1...",0,0,0,0
3230,Candler,"POLYGON ((-82.25457 32.35150, -82.25276 32.353...",0,0,0,0
3231,Tom Green,"POLYGON ((-101.26763 31.55646, -101.25039 31.5...",0,0,0,0
3232,Licking,"POLYGON ((-82.78181 39.94698, -82.78126 39.955...",0,0,0,0


### Exporting it as a shapefile

In [19]:
df_counties.to_file(DATA_DIR + '/hh_num_counties.shp')

In [20]:
df = gpd.read_file(DATA_DIR + '/hh_num_counties.shp')
df

,county,hh_half_mi,hh_one_mi,hh_five_mi,hh_total,geometry
0,Riley,0,0,0,0,"POLYGON ((-96.96095 39.28670, -96.96106 39.288..."
1,Ringgold,0,0,0,0,"POLYGON ((-94.47167 40.81255, -94.47166 40.819..."
2,Carbon,0,0,0,0,"POLYGON ((-109.79867 45.16734, -109.68779 45.1..."
3,Bear Lake,0,0,0,0,"POLYGON ((-111.63452 42.57034, -111.63010 42.5..."
4,Buffalo,0,0,0,0,"POLYGON ((-92.08384 44.41200, -92.08310 44.414..."
...,...,...,...,...,...,...
3103,Asotin,0,0,0,0,"POLYGON ((-117.47999 46.12199, -117.41948 46.1..."
3104,Candler,0,0,0,0,"POLYGON ((-82.25457 32.35150, -82.25276 32.353..."
3105,Tom Green,0,0,0,0,"POLYGON ((-101.26763 31.55646, -101.25039 31.5..."
3106,Licking,0,0,0,0,"POLYGON ((-82.78181 39.94698, -82.78126 39.955..."
